
# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.

+ Load the environment variables using dotenv. (1 pt)

In [6]:
# Write your code below.
import pandas as pd 
from dotenv import load_dotenv
load_dotenv()


True

In [7]:
import dask.dataframe as dd

c:\Users\zinth\anaconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:17: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(
C:\Users\zinth\AppData\Local\Temp\ipykernel_7752\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be r

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob

# Write your code below.
# Load the PRICE_DATA environment variable
price_data_path = os.getenv('PRICE_DATA')

# Find all Parquet files in the PRICE_DATA directory
parquet_files = glob(os.path.join(price_data_path, '*.parquet'))

# Load the Parquet file into a Dask DataFrame
dd_df = dd.read_parquet(parquet_files)

print (price_data_path)
print (parquet_files)
    

../05_src/data/prices/
['../05_src/data/prices\\example.parquet']


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [10]:
# Write your code below.

# Add lags for the Close and Adj_Close variables
dd_df['Close_lag'] = dd_df['Close'].shift(1)
dd_df['Adj_Close_lag'] = dd_df['Adj_Close'].shift(1)

# Add returns based on Adjusted Close
dd_df['returns'] = (dd_df['Adj_Close'] / dd_df['Adj_Close_lag']) - 1

dd_df['hi_lo_range'] = dd_df['High'] - dd_df['Low']

# Assign the result to dd_feat
dd_feat = dd_df


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [63]:
# Write your code below.
# Convert the Dask DataFrame to a pandas DataFrame
pandas_df = dd_df.compute()

# Add a rolling average return calculation with a window of 10 days
pandas_df['rolling_avg_return'] = pandas_df['returns'].rolling(window=10).mean()

# Show the resulting DataFrame
print(pandas_df)


  ticker  Close  Adj_Close  Close_lag  Adj_Close_lag    returns  \
0   AAPL    150        149        NaN            NaN        NaN   
1  GOOGL   2800       2790      150.0          149.0  17.724832   
2   MSFT    300        295     2800.0         2790.0  -0.894265   

   rolling_avg_return  
0                 NaN  
1                 NaN  
2                 NaN  


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

1.Yes, in this case, it was necessary to convert to a Pandas DataFrame to calculate the moving average return using the .rolling(10).mean() function. Dask DataFrames do not support the same operations as Pandas DataFrames, particularly for rolling operations across multiple partitions.

2. Ideally, it would be better to perform the rolling average calculation using Dask, especially if dealing with large datasets that do not fit into memory. Dask is designed to handle larger-than-memory computations efficiently, enabling you to leverage parallel processing. However, as of now, Dask has limited support for certain operations compared to Pandas, making some calculations easier to execute after converting to a Pandas DataFrame.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.